# Path Operations

## Metodo GET

No solo podemos retornar strings, sino cualquier otra cosa, como una cadena de texto, un diccionario, o codigo *html*:

Importar la clase *HTMLResponse*

```python
from fastapi.responses import HTMLResponse
...
...
return HTMLResponse('<h1>Hello World</h1>')
```

### Creando un segundo endpoint

```python
@app.get('/movies', tags=['movies'])
def get_movies():
    return movies
```

Donde movies es un diccionario


## Parametros de ruta

Se modifico el codigo para dejar los datos de todas las peliculas en formato Json en un archivo aparte.

![](https://i.imgur.com/V3TWQE0.png)

Para implementarlo:

```python
@app.get('/movies/{id}', tags=['movies'])
def get_movie(id: int):
    for item in movies:
        if item['id'] == id:
            return item
    return None
```

- El parametro *tags* no es necesario para que funcione

### testiando

caando el parametro es 10.

    http://127.0.0.1:5000/movies/10

![](https://i.imgur.com/iYQivM0.png)

## Parametros Query

Son una serie de valores *clave:valor* para extender la funcionalidad de las busquedas. Implementaremos un filtrado de peliculas por su categoria. 

### Primera Implemenacion

```python
@app.get('/movies/', tags=['movies'])👈
def get_movies_by_category(category: str):
    return category
```

- Simplemente añadimos el parametro en la funcion, y no lo especificamos en la *url* como lo hizimos en los parametros de ruta, FastAPI va detectarlo como un *parametro Query*

- Asegurate de colocar el */* despues de *movies*, sino reescribira la ruta

        @app.get('/movies', tags=['movies']) 👈 La rescribira

Veamos en la documentacion lo que tenemos hasta ahora:

![](https://i.imgur.com/fYrerG9.png)

### Testiando:

![](https://i.imgur.com/oNQanYq.png)

En la seccion de parametros, me esta indicando requiere un parametro llamado *category*, es requerido, y es de tipo *query*

![](https://i.imgur.com/YnGxNOW.png)

Observa los parametro query van al final de la URL, despues del signo de interrogacion. Con su clave y su valor: 

    "category" : "western"

### Agregando mas parametros query

Añadimos el año:

```python
@app.get('/movies/', tags=['movies'])
def get_movies_by_category(category: str, year: int):
    return category
```

![](https://i.imgur.com/gBucadV.png)

- La *Request URL* ahora tendra dos datos de tipo llave:valor separados por un &

        http://127.0.0.1:5000/movies/?category=western&year=1994

### Implementacion Final

Para que muestre el listado de las peliculas por su categoria, el año lo voy a ignorar:

```python
@app.get('/movies/', tags=['movies'])
def get_movies_by_category(category: str, year: int):
    return [movie for movie in movies if movie['category'] == category]
```

Como vez es una list comprehension.

## Metodo POST

Para crear una nueva pelicula

![](https://i.imgur.com/ikSTRBr.png)

### Primera implementacion

```python
@app.post('/movies', tags=['movies'])
def create_movie(id: int, title: str, year: int, rating: float, category: str):
    return id, title, year
```

En la documentacion ya aparece el metodo *POST* en el endpoint *movies*

![](https://i.imgur.com/hmM11j3.png) 

Sin embargo, en la seccion de parametros esta requiriendo todos esos valores como parametros, porque los añadimos como parametros en la funcion, pero esta es una mala implementacion.

![](https://i.imgur.com/tO9UBhH.png)


### Segunda Implementacion

Vamos hacer que todos estos datos, clave:valor, lleguen en el cuerpo de la peticion. 

- Importar la clase *Body*.

        from fastapi import FastAPI, Body

- Al decirle que cada parametro es igual a Body, indicara que hacen parte del contenido de la peticion

```py
@app.post('/movies', tags=['movies'])
def create_movie(id: int = Body(), 
                 title: str = Body(), 
                 year: int = Body(), 
                 rating: float = Body(), 
                 category: str= Body()):
    
    new_movie = dict(id = id,
                     title = title,
                     year = year,
                     rating = rating,
                     category = category)
    
    movies.append(new_movie)
    
    return [movie for movie in movies if movie['id'] == id]
```

![](https://i.imgur.com/G63kv0R.png)

Y ahora ves, no esta pidiendo valores como parametros, sino un objeto que contiene cada uno de dichos valores. 



### Reto

Implementar, PUT(update) y DELETE

```py
@app.put('/movies/{id}', tags=['movies'])
def update_movie(id: int, 
                 title: str = Body(), 
                 year: int = Body(), 
                 rating: float = Body(), 
                 category: str= Body()):
    
    for movie in movies:
        if movie['id'] == id:
            
            movie['title'] = title
            movie['year'] = year
            movie['rating'] = rating
            movie['category'] = category
            break
    
    return id, title, year, rating, category
```



Y ahora delete:

```py
@app.delete('/movies/{id}', tags=['movies'])
def delete_movie(id: int):
    for movie in movies:
        if movie["id"] == id:
            movies.remove(movie)
            break
    return id
```